# Hybrid Approach -- Scenario II

In this notebook, we report the code related to the *hybrid* appraoch in our paper [[1]](#ourpaper). We consider the delay caused by the feedback loop used by the mobile user equipment to send to the base station the channel state information (CSI). Due to this feedback delay, the CSI available at the base station becomes outdated. In our paper, we propose a hybrid *data-driven* and *model-based* approach. The model-based part consists of using a FIR Wiener filter to predict the instantaneous CSI given the channel history, which we assume available at the base station. The data-driven part consists of a neural network whose input is the output of the Wiener filter. Ths network maps the CSI to the probability of an error event for all the MCSs and selects the MCS that maximizes the spectral efficiency

Under Scenario II, we implement a different Wiener filter for each delay, but we design it to work on the full range of dopplers and signal-to-noise ratios. We then build a different neural netwok for each of Wiener's outputs and we train each neural network on the full range of dopplers and signal-to-noise ratios.

It must be noted that the training datasets listed below in the code are currently not available in the repository due to space limitations. The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Trained_folder_ScenarioII*. At any rate, in the repository, the reader can also find the codes in *radio_data* folder which can be run to generate the datasets. 

**Note**: the training might take some hours, depending on the available computational resources, the dimension of the training set, the dimension of the network, and the number of epochs. 


<a id='ourpaper'></a> [1] "Wireless link adaptation - a hybrid data-driven and model-based approach", Lissy Pellaco, Vidit Saxena, Mats Bengtsson, Joakim Jaldén. Submitted to WCNC 2020.

## Import libraries and utility functions

In [ ]:
import numpy as np
import time
from keras.optimizers import Adam
import utilities as utils
from keras.backend.tensorflow_backend import set_session
from keras.backend import clear_session
import tensorflow as tf


In [ ]:
# Number of Wiener filter taps
Wiener_filter_taps = 10
# Flag used to indicate if the channel is noisy
CHANNEL_EST_NOISE = True
# Parameters related to neural network training
BATCH_SIZE = 32
NROF_EPOCHS = 10
TRAINING_FRACTION = 0.2

# Flag to indicate if the trained models should be saved
save_model = False

## Load the Dataset

The channel dataset is a dict with the following keys :  
 - 'channel'
     - Complex channel coefficients 
     - Numpy array [ NROF_FRAMES x NROF_SUBCARRIERS x NROF_SNRS]
 - 'block_success'
      - Binary success events (ACKs)
      - Numpy array [ NROF_FRAMES x NROF_MCS x NROF_SNRS]
 - 'snrs_db '      
     - Evaluated average SNR values
     - Numpy array [ NROF_SNRS ]
 - 'block_sizes'
     - Evaluated transport block sizes
     - Numpy array [ NROF_MCS ]
     
The name of the dataset, e.g., ITU_VEHICULAR_B_5000_60kmph, is to be interpreted in this way: 
 - channel model (ITU_VEHICULAR_B)
 - number of channel realizations (5000)
 - relative velocity between the base station and the user mobile equipment (60kmph)
 
The **training datasets can be found at**: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Trained_folder_ScenarioII*

In [ ]:
# The files stored in the file_set ARE NOT in the repository due to space limitations.
# The training datasets can be found at: https://kth.box.com/s/tcd7y7rg3yau75kctw3regmyns8kfkr6 in the folder *Datasets*
# The reader has also access to the "radio_data/Generate_Data.ipynb" which we used to generate the training datasets.

FADING_CHANNEL_DATAFILES = []

FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_30kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_45kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_60kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_75kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_90kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_105kmph.npy')
FADING_CHANNEL_DATAFILES.append('Datasets/ITU_VEHICULAR_B_5000_120kmph.npy')

# Doppler frequency related to the datasets in "file_set"
dopplers_set = (20.0 / 3.0) * np.array([8.3,12.5,16.67,20.83,25,29.16,33.33])

# Maximum feedback delay that we consider
maximum_delay = 9

# Snrs range that we consider  
snrs_set = np.array(range(0,30))

channel_sampling_interval = 0.001

# Number of Wiener filter taps
Wiener_filter_taps = 10

## Wiener filter averaged over SNRs and Dopplers


In [ ]:
def create_ann_model(NROF_SUBCARRIERS, NROF_MCS):
        from keras.models import Sequential
        from keras.layers import Dense, Dropout

        model = Sequential()
        model.add( Dense( 1024, 
                          input_dim = NROF_SUBCARRIERS * 2, 
                          kernel_initializer='normal', 
                          activation='relu' ) )

        model.add( Dense( 512, 
                          kernel_initializer = 'normal', 
                          activation='relu' ) )

        model.add( Dense( 1024, 
                          kernel_initializer = 'normal', 
                          activation='relu' ) )

        model.add( Dense( NROF_MCS, 
                          kernel_initializer='normal', 
                          activation='sigmoid' ) )

        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # for binary classification


        return model


## Build the neural network model, define the optimizer, and the cost function

In [ ]:
def apply_wiener_filtering_average( Wiener_coeff,freq_domain_channel, snrs_dB, doppler_freq, delay = 1, N = 10 ):
    nrof_samples, nrof_subcarriers, nrof_snrs = freq_domain_channel.shape
    
    filtered_channel_freq_response = np.ndarray( freq_domain_channel.shape, dtype = np.complex128 )
   
    for snr_index in range( nrof_snrs ):    
    
        snr = 10 ** ( 0.1 *snrs_dB[ snr_index ] )

        channel_sampling_interval = 0.001
        

        for subc_index in range( nrof_subcarriers ):
            
            subcarrier_response = freq_domain_channel[:, subc_index, snr_index]
        
            #Apply the Wiener filter
            filtered_subc_response = np.convolve( Wiener_coeff, subcarrier_response, "full")
            filtered_channel_freq_response[ :, subc_index, snr_index ] = filtered_subc_response[ : -N + 1 ]
            
    return filtered_channel_freq_response


## Train the Neural Network on Wiener filter's output

In [ ]:
for DELAY in range(0, maximum_delay + 1):

    Wiener_coeff=np.zeros(Wiener_filter_taps)
    Wiener_coeff_temp=np.zeros(Wiener_filter_taps)

    for doppler_freq in dopplers_set:

        for snr_index in snrs_set:    

            snr = 10 ** ( 0.1 *snrs_dB[ snr_index ] )
            

            autocorrelation_of_reference = utils.autocorrelation( np.arange(0,Wiener_filter_taps,1),
                                                                          doppler_freq,
                                                                          channel_sampling_interval )


            crosscorrelation = utils.autocorrelation( np.arange(DELAY, Wiener_filter_taps + DELAY, 1),
                                                              doppler_freq,
                                                              channel_sampling_interval )

            Wiener_coeff = Wiener_coeff + utils.Wiener_filter_coeff_scaled( autocorrelation_of_reference,
                                                                     crosscorrelation,
                                                                     DELAY,
                                                                     Wiener_filter_taps,
                                                                     snr,
                                                                     True,
                                                                     doppler_freq,
                                                                     channel_sampling_interval )

    # Averaged Wiener filter over the snrs and dopplers
    Wiener_coeff = Wiener_coeff/(210.0)
    ######################################################################################################
    channel_coeff  = []
    block_success  = []

    # Extract training data from datasets
    for file_index, file in enumerate( FADING_CHANNEL_DATAFILES ):
        DATASET = np.load( file, allow_pickle = True )[()]

        nrof_train_samples = int( TRAINING_FRACTION * DATASET['channel'].shape[0] )
        coeff = utils.calculate_channel_coefficients_scaled( DATASET['channel'][ :nrof_train_samples, :, : ],
                                                             DATASET['snrs_db'],
                                                             channel_estimation_noise = CHANNEL_EST_NOISE )

        coeff_filtered = apply_wiener_filtering_average( Wiener_coeff,coeff, 
                                                 DATASET[ 'snrs_db' ], 
                                                 doppler_freq = dopplers_set[file_index], 
                                                 delay = DELAY, 
                                                 N = 10 )

        channel_coeff.append( coeff_filtered )
        block_success.append( DATASET['block_success'][ :nrof_train_samples, :, : ] )

    channel_coeff = np.vstack( channel_coeff )
    block_success = np.vstack( block_success )
    BLOCK_SIZES = DATASET[ 'block_sizes' ] 
    SNRS_DB     = DATASET['snrs_db']
    NROF_FRAMES, NROF_SUBCARRIERS, NROF_SNRS = channel_coeff.shape
    NROF_MCS = block_success.shape[ 1 ]
    
    channel_coeff_concat = np.concatenate( ( np.real( channel_coeff ), np.imag( channel_coeff ) ), axis = 1 )
    if DELAY > 0:
        train_input  = flatten_snr_axis( channel_coeff_concat[ :-DELAY, :, : ] )
    else:
        train_input  = flatten_snr_axis( channel_coeff_concat )

    train_target = flatten_snr_axis( block_success [ DELAY :, :, : ] )

    train_input, train_target = utils.shuffle_data( train_input, train_target )

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True  

    sess = tf.Session( config = config )
    set_session(sess) 

    
    model = create_ann_model(NROF_SUBCARRIERS, NROF_MCS)

    history = model.fit( train_input, 
                         train_target, 
                         batch_size = BATCH_SIZE, 
                         epochs     = NROF_EPOCHS, 
                         validation_split = 0.1, 
                         verbose    = 1 ) # the "verbose parameter" can be changed to display more about the training progess of each epoch

    file = 'Trained_models_ScenarioII/ANN_MCS_PRED_WIENER_FILT_COEFF_AVERAGE_DELAY_%d.h5'%(DELAY)
    if save_model == True:
        model.save( file )
        print( 'Saved model to %s'%( file ) )